In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut, cross_val_score, KFold
from sklearn.metrics import accuracy_score

# ---------------------------
# 1. 加载数据并预处理
# ---------------------------
data = pd.read_csv("diabetes_dataset_1.csv")

# 初始化增强数据集：先用所有唯一样本作为基础
aug_data = data.copy().reset_index(drop=True)

# 为每个唯一样本建立复制计数（最多允许复制2次，可根据论文中描述调整）
max_duplications = 2
duplication_counts = {i: 0 for i in data.index}

models = {
    "Naïve Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "K-NN": KNeighborsClassifier(n_neighbors=11),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "SVM": SVC(random_state=42),
    "MLP": MLPClassifier(hidden_layer_sizes=(24, 8), random_state=42, max_iter=1000)
}

clf = models.get("Decision Tree")

# ---------------------------
# 2. 定义留一交叉验证函数
# ---------------------------
def loo_cv_accuracy(dataset):
    """
    对给定数据集使用留一交叉验证（LOO-CV）训练决策树，并返回整体准确率及每个样本的预测结果
    """
    X = dataset.drop(columns=['Outcome'])
    y = dataset['Outcome']
    loo = LeaveOneOut()
    predictions = []
    # 按照数据集的顺序依次进行 LOO-CV
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        predictions.append(pred[0])
    acc = accuracy_score(y, predictions)
    return acc, predictions

# ---------------------------
# 3. 迭代复制误分类样本生成增强数据集
# ---------------------------
max_rounds = 10         # 最大迭代轮数
prev_acc = 0            # 上一轮的 LOO-CV 准确率
round_num = 0

while round_num < max_rounds:
    acc, predictions = loo_cv_accuracy(aug_data)
    print(f"第 {round_num} 轮，LOO-CV 准确率：{acc:.4f}")
    # 如果准确率没有明显提升，则停止复制
    if acc - prev_acc < 1e-4:
        print("准确率无明显提升，停止数据增强。")
        break
    prev_acc = acc
    # 从增强数据集中挑选出原始唯一样本（data）的误分类样本
    # 注意：aug_data 中有重复的样本，我们只考虑那些在 data 中的记录
    misclassified_indices = []
    # 使用 aug_data 原有顺序逐个检查（假设 data 的索引为 0~N-1）
    for idx, (pred, true_val) in enumerate(zip(predictions, aug_data['Outcome'])):
        # 如果该样本属于原始唯一样本（索引小于 data.shape[0]）且被误分类，则记录其在 data 中的索引
        if idx < data.shape[0] and pred != true_val:
            misclassified_indices.append(idx)
    if len(misclassified_indices) == 0:
        print("没有发现误分类的原始样本，停止数据增强。")
        break

    # 对每个误分类的唯一样本，如果复制次数未达到上限，则复制一次
    duplicates_to_add = []
    for idx in misclassified_indices:
        if duplication_counts[idx] < max_duplications:
            duplication_counts[idx] += 1
            # 注意这里直接从 data 中提取原始记录（保持原始数据不变）
            row = data.loc[[idx]]
            duplicates_to_add.append(row)
    if len(duplicates_to_add) == 0:
        print("所有误分类样本的复制次数已达上限，停止数据增强。")
        break

    # 将复制的样本追加到增强数据集中
    duplicates_df = pd.concat(duplicates_to_add, ignore_index=True)
    aug_data = pd.concat([aug_data, duplicates_df], ignore_index=True)
    print(f"本轮复制了 {duplicates_df.shape[0]} 个样本，增强数据集大小：{aug_data.shape[0]}")
    round_num += 1

print("最终增强数据集大小：", aug_data.shape[0])

# ---------------------------
# 4. 利用10折交叉验证评估模型性能
# ---------------------------
def evaluate_model(dataset):
    """
    利用10折交叉验证评估决策树模型在给定数据集上的平均准确率
    """
    X = dataset.drop(columns=['Outcome'])
    y = dataset['Outcome']
    cv = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    return scores.mean()

orig_acc = evaluate_model(data)
aug_acc = evaluate_model(aug_data)

print("\n使用10折交叉验证评估结果：")
print(f"原始数据集准确率：{orig_acc*100:.2f}%")
print(f"增强数据集准确率：{aug_acc*100:.2f}%")


第 0 轮，LOO-CV 准确率：0.7031
本轮复制了 228 个样本，增强数据集大小：996
第 1 轮，LOO-CV 准确率：0.8936
本轮复制了 106 个样本，增强数据集大小：1102
第 2 轮，LOO-CV 准确率：0.9664
本轮复制了 37 个样本，增强数据集大小：1139
第 3 轮，LOO-CV 准确率：0.9824
本轮复制了 20 个样本，增强数据集大小：1159
第 4 轮，LOO-CV 准确率：0.9750
准确率无明显提升，停止数据增强。
最终增强数据集大小： 1159

使用10折交叉验证评估结果：
原始数据集准确率：71.10%
增强数据集准确率：93.70%
